<a href="https://colab.research.google.com/github/SolovevDR/NIR/blob/main/ParserAccsessLog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Подготовка среды для работы(запускать строго в этом порядке, не будет иначе geoip работать)

In [1]:
pip install python-geoip-geolite2

In [2]:
pip install python-geoip-python3

In [3]:
pip install user_agents

In [4]:
pip install apachelogs

In [5]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/NIR')



---
---
---



In [6]:
import time 
import pandas as pd
from urllib.parse import urlparse
import re
from dateutil.parser import parse as dateparser
from user_agents import parse
from geoip import geolite2
import datetime
from apachelogs import LogParser

In [7]:
def ban_separation_date_time(datetim):
    req_requset = re.compile(r'([0-9]+.[0-9]+.[0-9]+).([0-9]+:[0-9]+:[0-9]+.[0-9]+).(.[0-9]+)')
    date_and_time_list = []
    for i in range(len(datetim)):
        result = req_requset.match(datetim[i])
        date_and_time = result.group(1) + ' ' + result.group(2) + result.group(3)

        dt = dateparser(date_and_time)
        date_and_time_list.append(dt)
    return  date_and_time_list

a = ban_separation_date_time(['2021-10-05 22:51:16.972634 +0000 UTC'])

In [8]:
def create_dict(new_date_list, status_list, ip_list):
    ban_dict = {}
    for i in range(len(new_date_list)):
        if (status_list[i] == 'added'):
            if ban_dict.get(ip_list[i]) == None: 
                ban_dict[ip_list[i]] = [new_date_list[i]]
            else: 
                spis = ban_dict[ip_list[i]]
                spis.append(new_date_list[i])
                ban_dict[ip_list[i]] = spis
        if (status_list[i] == 'removed') and (ban_dict.get(ip_list[i]) != None):
                spis = ban_dict[ip_list[i]]
                spis.append(new_date_list[i])
                ban_dict[ip_list[i]] = spis
    keys_list = list(ban_dict.keys())
    for i in keys_list:
        time_ban_unban = ban_dict.get(i)
        if len(time_ban_unban)%2 != 0:
            b = time_ban_unban[len(time_ban_unban)-1]
            b = b + datetime.timedelta(seconds=30)
            spis = ban_dict[i]
            spis.append(b)
            ban_dict[i] = spis
    return(ban_dict)


In [9]:
df = pd.read_csv('ban_unban_table.csv')

date_list = df['created_at'].tolist()
status_list = df['action'].tolist()
ip_list = df['ip'].tolist()

new_date_list = ban_separation_date_time(date_list)

print(new_date_list)
print(status_list)
print(ip_list)

[datetime.datetime(2021, 10, 28, 20, 13, 0, 494701, tzinfo=tzlocal()), datetime.datetime(2021, 10, 28, 20, 12, 30, 510202, tzinfo=tzlocal()), datetime.datetime(2021, 10, 28, 20, 12, 30, 510202, tzinfo=tzlocal()), datetime.datetime(2021, 10, 28, 20, 12, 30, 510202, tzinfo=tzlocal()), datetime.datetime(2021, 10, 28, 20, 12, 30, 510202, tzinfo=tzlocal()), datetime.datetime(2021, 10, 28, 20, 12, 30, 510202, tzinfo=tzlocal()), datetime.datetime(2021, 10, 28, 20, 14, 0, 523427, tzinfo=tzlocal()), datetime.datetime(2021, 10, 28, 20, 14, 0, 523427, tzinfo=tzlocal()), datetime.datetime(2021, 10, 28, 20, 14, 0, 523427, tzinfo=tzlocal()), datetime.datetime(2021, 10, 28, 20, 14, 0, 523427, tzinfo=tzlocal()), datetime.datetime(2021, 10, 28, 20, 12, 0, 500445, tzinfo=tzlocal()), datetime.datetime(2021, 10, 28, 20, 12, 0, 500445, tzinfo=tzlocal()), datetime.datetime(2021, 10, 28, 20, 12, 0, 500445, tzinfo=tzlocal()), datetime.datetime(2021, 10, 28, 20, 15, 30, 488352, tzinfo=tzlocal()), datetime.date

In [10]:
ban_dict = create_dict(new_date_list, status_list, ip_list)

In [11]:
def find_info_with_ip(ip):

    m = geolite2.lookup(ip)

    try:
        country = m.country
    except:
        country = "can't find country"

    try:
        continent = m.continent
    except:
        continent = "can't find continent"

    try:
        timezone = m.timezone
    except:
        timezone = "can't find timezone"

    return country, continent, timezone

country, continent, timezone = find_info_with_ip('17.0.0.1')

In [12]:
def separation_date_time(datetim):
    #req_requset = re.compile(r'.([0-9]+.[0-9]+.[0-9]+).([0-9]+:[0-9]+:[0-9]+)(.[0-9]+.[0-9]+)')

    #result = req_requset.match(datetime)
    #date_and_time = result.group(1) + ' ' + result.group(2)

    dt = dateparser(datetim)
    new_datetime = dt.astimezone()
    return  new_datetime.date(), new_datetime.time(), new_datetime

date, time, new_datetime = separation_date_time('2019-01-22 03:56:14+03:30')
print(new_datetime)

2019-01-22 00:26:14+00:00


In [13]:
def find_info_with_request(request):

    url_scheme = None
    url_netloc = None
    url_path = None
    url_params = None
    url_query = None
    url_fragment = None
    
    try:
        method, url, http = request.split()

        url_line = urlparse(url)

        if url_line.scheme == '':
            url_scheme = 'no url scheme'
        else:
            url_schem = url_line.scheme

        if url_line.netloc == '':
            url_netloc = 'no url netloc'
        else:
            url_netloc = url_line.netloc

        if url_line.path == '':
            url_path = 'no url path'
        else:
            url_path = url_line.path

        if url_line.params == '':
            url_params = 'no url params'
        else:
            url_params = url_line.params

        if url_line.query == '':
            url_query = 'no url query'
        else:
            url_query = url_line.query

        if url_line.fragment == '':
            url_fragment = 'no url fragment'
        else:
            url_fragment = url_line.fragment
  
    except:
        method = 'error in request'
        url_scheme = 'error in request'
        url_netloc = 'error in request'
        url_path = 'error in request'
        url_params = 'error in request'
        url_query = 'error in request'
        url_fragment = 'error in request'
        http = 'error in request'

    return method, url_scheme, url_netloc, url_path, url_params, url_query, url_fragment, http

method, url_scheme, url_netloc, url_path, url_params, url_query, url_fragment, http = find_info_with_request('GET /image/60844/productModel/200x200 HTTP/1.1')

In [14]:
def find_info_with_Useragent(useragent_line):
    req_request = re.compile(r'([a-zA-z]+).')

    user_agent = parse(useragent_line)

    try:
        os = user_agent.os.family
    except:
        os = 'no information'

    try:
        browser = user_agent.browser.family
    except:
        browser = 'no information'

    try:
        device = user_agent.device.family
    except:
        device = 'no information'

    try:
        result = req_request.match(useragent_line)
        exit_system = result.group(1)
    except:
        exit_system = 'no information'

    return exit_system, os, browser, device

find_info_with_Useragent('Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build/HuaweiALE-L21) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.158 Mobile Safari/537.36')

('Mozilla', 'Android', 'Chrome Mobile', 'ALE-L21')

In [15]:
def copy_size_return_object(size_object):


    try:
        new_size_return_object = int(size_object)
    except:
        new_size_return_object = 0

    return new_size_return_object

copy_size_return_object(5667)

5667

In [16]:
status_request_list = []

In [29]:
def markup(datetim, ip, ban_dict, number):
    keys_list = list(ban_dict.keys())
    if ip in keys_list:
        print(ip)
        spis = ban_dict[ip]
        if (datetim > spis[0] - datetime.timedelta(seconds=30)) and (datetim < spis[1]):
            spis.pop(0)
            spis.pop(1)
            ban_dict[ip] = spis
            print(number)
            return 0 
        else:
            return 1 
    else:
        return 1
    

In [18]:
#col_names = ['country', 'continent', 'timezone', 'date', 'time', 'method', 'url_sheme', 'url_netloc', 'url_path', 'url_params', 'url_querty', 'url_fragment', 'http', 'status_code', 'size_object', 'exit_system', 'os', 'browser', 'device']
my_df = pd.DataFrame()


In [30]:
parser2 = LogParser('%h %l %u %t \"%r\" %>s %b \"%{Referer}i\" \"%{User-Agent}i\"')

ip_list = []
country_list = []
continent_list = []
timezone_list = []
date_list = []
time_list = []
datetime_list = []
method_list = []
url_scheme_list = []
url_netloc_list = []
url_path_list = []
url_params_list = []
url_query_list = []
url_fragment_list = []
http_list = []
size_return_object_list = []
exit_system_list = []
os_list = []
browser_list = []
device_list = []
status_code_list = []
status_request_list = []
number = 0




with open('my.srvgame.ru.access.log') as fp:  # doctest: +SKIP
    for entry in parser2.parse_lines(fp):
        country, continent, timezone = find_info_with_ip(str(entry.remote_host))
        date, time, datetim = separation_date_time(str(entry.directives['%t']))
        method, url_scheme, url_netloc, url_path, url_params, url_query, url_fragment, http = find_info_with_request(str(entry.request_line))
        status_code = entry.directives['%>s']
        size_return_object = copy_size_return_object(entry.directives['%b'])
        exit_system, os, browser, device = find_info_with_Useragent(str(entry.directives['%{User-Agent}i']))
        mark = markup(datetim, str(entry.remote_host), ban_dict, number)

        ip_list.append(str(entry.remote_host))
        status_code_list.append(status_code)
        country_list.append(country)
        continent_list.append(continent)
        timezone_list.append(timezone)
        date_list.append(date)
        time_list.append(time)
        datetime_list.append(datetim)
        method_list.append(method)
        url_scheme_list.append(url_scheme)
        url_netloc_list.append(url_netloc)
        url_path_list.append(url_path)
        url_params_list.append(url_params)
        url_query_list.append(url_query)
        url_fragment_list.append(url_fragment)
        http_list.append(http)
        size_return_object_list.append(size_return_object)
        exit_system_list.append(exit_system)
        os_list.append(os)
        browser_list.append(browser)
        device_list.append(device)
        status_request_list.append(mark)
        number = number + 1



my_df['ip'] = ip_list
my_df['country'] = country_list
my_df['continent'] = continent_list
my_df['timezone'] = timezone_list
my_df['datetime'] = datetime_list
my_df['date'] = date_list
my_df['time'] = time_list
my_df['method'] = method_list
my_df['url_sheme'] = url_scheme_list
my_df['url_netloc'] = url_netloc_list
my_df['url_path'] = url_path_list
my_df['url_params'] = url_params_list
my_df['url_querty'] = url_query_list
my_df['url_fragment'] = url_fragment_list
my_df['http'] = http_list
my_df['status_code'] = status_code_list
my_df['size_object'] = size_return_object_list
my_df['exit_system'] = exit_system_list
my_df['os'] = os_list
my_df['browser'] = browser_list
my_df['device'] = device_list
my_df['markup'] = status_request_list
my_df.to_csv('DICT_FOR_LEARN.csv', header=True, sep=';')


46.191.138.41
46.191.138.41
46.191.138.41
46.191.138.41
46.191.138.41
46.191.138.41
46.191.138.41
46.191.138.41
46.191.138.41
46.191.138.41


In [20]:
print(my_df['datetime'])

0       2021-10-28 03:26:08+00:00
1       2021-10-28 03:29:32+00:00
2       2021-10-28 03:29:32+00:00
3       2021-10-28 03:29:33+00:00
4       2021-10-28 03:29:33+00:00
                   ...           
15129   2021-10-29 03:19:05+00:00
15130   2021-10-29 03:19:29+00:00
15131   2021-10-29 03:20:40+00:00
15132   2021-10-29 03:22:06+00:00
15133   2021-10-29 03:22:10+00:00
Name: datetime, Length: 15134, dtype: datetime64[ns, UTC]


In [22]:
print(df.columns)

Index(['instance_id', 'instance_name', 'created_at', 'action', 'ip', 'source',
       'country', 'city', 'as_number', 'as_name'],
      dtype='object')
